# Urban PointCloud Processing

This notebook shows a "complete solution" in which a single point cloud file is automatically labeled using the various tools available in this repository. For clarity we skip [preprocessing of AHN data](1.%20AHN%20preprocessing.ipynb) and assume all necassary data files are already available.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import time

import src.fusion as fusion
import src.region_growing as growing
from src.pipeline import Pipeline
from src.utils.labels import Labels

In [ ]:
# Select the file to process.
in_file = '../datasets/pointcloud/filtered_2386_9702.laz'
out_file = '../datasets/pointcloud/complete_2386_9702.laz'

## Data Fusion

First, we use data fusion to automatically label ground and building points.

For details and more options, see [2. Data fusion](2.%20Data%20fusion.ipynb).

In [ ]:
# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'

# Ground fuser using pre-processed AHN data.
npz_ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_data_folder,
                                   method='npz', target='ground', epsilon=0.2)
# Building fuser using pre-processed AHN data.
npz_building_fuser = fusion.AHNFuser(Labels.BUILDING, ahn_data_folder,
                                     method='npz', target='building', epsilon=0.2)

## Region Growing

Then, we use region growing to refine the buildings. This will make sure that protruding elements such as balconies are labelled correctly.

For details, installation instructions and more options, see [3. Region growing](3.%20Region%20growing.ipynb).

In [ ]:
# Region growing with building points as initial seed points. 
exclude_labels = (Labels.GROUND,)
building_grower = growing.LabelConnectedComp(Labels.BUILDING, exclude_labels,
                                             octree_level=10, min_component_size=100)

## Process the file

In [ ]:
# Set-up pipeline.
fusers = (npz_ground_fuser, npz_building_fuser)
growers = (building_grower,)
pipeline = Pipeline(fusers, growers)

start = time.time()
# Process the file.
pipeline.process_file(in_file, out_file)
end = time.time()
print(f'Tile labelled in {end-start:.2f} seconds.')

## View the result (e.g. in CloudCompare)

The final result can be viewed in CloudCompare. Open the out_file (e.g. datasets/pointcloud/labelled_2386_9702.laz) and change `Colors` from "RGB" to "Scalar field".

The result should look like this:
![Demo result](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/raw/main/media/examples/demo_result.png)